In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
from scipy import stats
import math
%matplotlib inline
from sklearn import preprocessing
import statsmodels.formula.api as smf
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

pd.options.display.float_format = '{:.3f}'.format

# Crime Data

In [2]:
fbi = pd.read_csv('C:\\Users\\cocob\\Downloads\\table_82.csv', encoding = "ISO-8859-1")

## Basic Data Info

In [3]:
fbi.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1861.000,0.000,0.000,nan,0.000,0.000,0.000,12.000,2.000,10.000,0.000,0.000
1,Addison Town and Village,2577.000,3.000,0.000,nan,0.000,0.000,3.000,24.000,3.000,20.000,1.000,0.000
2,Akron Village,2846.000,3.000,0.000,nan,0.000,0.000,3.000,16.000,1.000,15.000,0.000,0.000
3,Albany,97956.000,791.000,8.000,nan,30.000,227.000,526.000,4090.000,705.000,3243.000,142.000,nan
4,Albion Village,6388.000,23.000,0.000,nan,3.000,4.000,16.000,223.000,53.000,165.000,5.000,nan


In [4]:
fbi.describe()

,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
count,348.000,348.000,348.000,0.000,348.000,348.000,348.000,348.000,348.000,348.000,348.000,187.000
mean,40037.632,201.595,1.566,nan,5.865,72.902,121.261,792.606,119.684,637.017,35.905,1.872
std,450037.368,2815.269,18.304,nan,60.425,1031.033,1706.132,7659.725,924.949,6346.054,403.424,10.693
min,526.000,0.000,0.000,nan,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,3003.000,2.000,0.000,nan,0.000,0.000,1.000,40.500,6.000,31.000,0.000,0.000
50%,7233.500,6.000,0.000,nan,0.000,1.000,4.000,112.500,17.500,94.000,2.000,0.000
75%,18427.500,22.000,0.000,nan,2.000,5.000,14.000,341.000,51.250,287.250,7.000,1.000
max,8396126.000,52384.000,335.000,nan,1112.000,19170.000,31767.000,141971.000,16606.000,117931.000,7434.000,132.000


## Data Transformation

In [5]:
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [6]:
fbi.rename(columns={'Murder and\nnonnegligent\nmanslaughter':'Murder', 'Property\ncrime': 'Property_crime'}, inplace=True)

In [7]:
fbi['Population^2'] = (fbi['Population'] ** 2)

In [8]:
fbi['Murder'] = np.where(fbi['Murder']>0, 1, 0)
fbi.head(5)

fbi['Robbery'] = np.where(fbi['Robbery']>0, 1, 0)
fbi.head(5)

,City,Population,Violent crime,Murder,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property_crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3,Population^2
0,Adams Village,1861.000,0.000,0,nan,0.000,0,0.000,12.000,2.000,10.000,0.000,0.000,3463321.000
1,Addison Town and Village,2577.000,3.000,0,nan,0.000,0,3.000,24.000,3.000,20.000,1.000,0.000,6640929.000
2,Akron Village,2846.000,3.000,0,nan,0.000,0,3.000,16.000,1.000,15.000,0.000,0.000,8099716.000
3,Albany,97956.000,791.000,1,nan,30.000,1,526.000,4090.000,705.000,3243.000,142.000,nan,9595377936.000
4,Albion Village,6388.000,23.000,0,nan,3.000,1,16.000,223.000,53.000,165.000,5.000,nan,40806544.000


In [21]:
fbi.columns

Index(['City', 'Population', 'Violent\ncrime', 'Murder',
       'Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2',
       'Robbery', 'Aggravated\nassault', 'Property_crime', 'Burglary',
       'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3', 'Population^2'],
      dtype='object')

In [27]:
fbi2 = fbi.drop(columns=['Rape\n(revised\ndefinition)1'])

In [36]:
fbi3 = fbi2.dropna()

In [37]:
data1=pd.DataFrame(fbi3, columns=['Population', 'Population^2', 'Murder', 'Robbery'])

In [39]:
regr = linear_model.LinearRegression()
Y = fbi3['Property_crime'].values.reshape(-1, 1)
X = fbi3[['Population', 'Population^2', 'Murder', 'Robbery']]
regr.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [40]:
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[1.93104508e-02 6.40075417e-08 1.77195340e+02 3.02481800e+01]]

Intercept: 
 [-41.99456913]

R-squared:
0.770049563947707
